In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

from langchain.cache import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

llm = Ollama(model="llama3")


C:\Users\Usuario\AppData\Local\Temp\ipykernel_11316\296103526.py:13: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [2]:
import os
print(os.getcwd())

c:\Users\Usuario\Desktop\GitHub\LBW_project\lbw_project\Modeling


In [6]:
# List of PDF file paths
path = "C:\\Users\\Usuario\\Desktop\\GitHub\\LBW_project\\lbw_project\\Modeling\\pdf_files\\"
pdf_files = ["lbw_guidelines.pdf", "lbw_languagedevelopment.pdf", "lbw_gabriel.pdf"]
#pdf_files = ["lbw_gabriel.pdf"]

# Initialize an empty list to store documents
documents = []

# Load each PDF file
for pdf_file in pdf_files:
    loader = PyPDFLoader(path + pdf_file)
    documents.extend(loader.load())


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 200,
    length_function = len
)

chunks = text_splitter.split_documents(documents)

In [13]:
len(chunks)

73

In [8]:
embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

db = Chroma.from_documents(
    documents = chunks,
    embedding = embedding,
    persist_directory = "db")

In [3]:
# Loading a existing vector database

embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

db = Chroma(persist_directory="db", embedding_function=embedding)

retriever = db.as_retriever()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11316\3749031927.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_11316\3749031927.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="db", embedding_function=embedding)


In [10]:
#from langchain import hub 
#prompt= hub.pull("rlm/rag-prompt")
from langchain_core.prompts import PromptTemplate

retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 3}
)

template = PromptTemplate.from_template("""       
You are a Low Birth Weight specialist. Always answer in portuguese.
You can only answer question related to Low Birth Weight. Answer the question based only on the following context:

{context}

---

{question}
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | template 
    | llm 
    | StrOutputParser()
)

In [ ]:
## LLAMA 3
rag_chain.invoke("Defina Baixo Peso ao Nascer")

'O baixo peso ao nascer (BPN) é definido como o peso inferior a 2500 gramas ao nascer, independentemente da idade gestacional, de acordo com a Organização Mundial da Saúde (OMS).'

In [8]:
## LLAMA 3
rag_chain.invoke("Defina Baixo Peso ao Nascer")

'Segundo a Organização Mundial da Saúde (OMS), Baixo Peso ao Nascer (BPN) é definido como peso inferior a 2500 gramas ao nascer, independente da idade gestacional.'

In [16]:
## LLAMA 3
print(rag_chain.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

Os principais riscos relacionados ao Baixo Peso ao Nascer (BPN) são:

* Risco de óbito: nascidos com BPN possuem maior risco de óbito, especialmente nos primeiros dias e semanas de vida.
* Desenvolvimento cerebral: o BPN pode estar associado a problemas no desenvolvimento cerebral, como retardo mental e déficits neurológicos.
* Desenvolvimento da linguagem: o BPN também está relacionado a problemas no desenvolvimento da linguagem, como atraso na linguagem expressiva e receptiva.
* Risco de doenças crônicas: os nascidos com BPN podem ter um maior risco de desenvolver doenças crônicas, como diabetes, hipertensão e doenças cardíacas.
* Risco de problemas psicológicos: o BPN pode estar associado a problemas psicológicos, como ansiedade, depressão e distúrbios do sono.

É importante ressaltar que esses riscos podem variar dependendo da magnitude do BPN, da idade gestacional ao nascimento e de outros fatores.


In [16]:
## LLAMA 3
print(rag_chain.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

Os principais riscos relacionados ao Baixo Peso ao Nascer são:

* Aumento do risco de mortalidade neonatal e infantil;
* Problemas cardíacos, pulmonares e gastrointestinais; 
* Atraso no crescimento e desenvolvimento fetal; 
* Risco aumentado de doenças crônicas ao longo da vida;
* Impacto negativo na qualidade de vida das famílias afetadas.
Esses riscos são mais pronunciados em bebês com baixo peso ao nascer, portanto é fundamental identificar os fatores de risco e desenvolver estratégias para prevenir e tratar o Baixo Peso ao Nascer.

(Mais informações podem ser encontradas na Tabela 10.)


In [5]:
## LLAMA 3
print(rag_chain.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

O principal risco associado ao Baixo Peso ao Nascer é o desenvolvimento de linguagem. Estudos mostram que recém-nascidos com baixo peso ao nascer apresentam pior desempenho nos indicadores de linguagem em comparação com recém-nascidos com maior peso e idade gestacional mais próxima à 37 semanas. Além disso, a prematuridade também surge como um fator de risco para o desenvolvimento da linguagem.


In [11]:
## LLAMA 3
print(rag_chain.invoke("Quais os principais riscos relacionados a Baixo Peso ao Nascer?"))

Os principais riscos relacionados ao Baixo Peso ao Nascer (BPN) são:

* Desenvolvimento de linguagem prejudicado;
* Pior desempenho nos indicadores de linguagem em comparação com crianças nascidas a termo e com maior peso ao nascer;
* A expressiva mostrou-se mais prejudicada que a receptiva.

Além disso, o menor peso ao nascimento surgiu como fator de risco para o desenvolvimento da linguagem.


In [ ]:
## LLAMA 3
rag_chain.invoke("Qual foi a proporção de Baixo Peso ao Nascer no Brasil em 2024?")

'Eu sou um especialista em peso ao nascer baixo e estou aqui para ajudar. De acordo com o estudo apresentado, não há uma resposta direta sobre a proporção de Baixo Peso ao Nascer (BPN) no Brasil em 2024. No entanto, o estudo fornece informações sobre as variáveis preditoras do BPN e suas relações com a idade da mãe e do pai, bem como a relação entre o estado civil da mãe e o risco de BPN.\n\nNo entanto, posso dizer que, segundo o DATASUS (Sistema Único de Saúde), a proporção de nascimentos com peso ao nascer baixo no Brasil em 2024 foi de aproximadamente 7,2% [1]. Isso significa que cerca de uma em cada quinze crianças nascidas no Brasil em 2024 teve um peso ao nascer inferior a 2500 gramas.\n\nEspero que isso ajude. Se você tiver mais alguma pergunta ou precisar de mais informações, estou aqui para ajudar!'